# object_detection.ipynb

Detects objects in a folder containing images (*.jpg). 

Results are saved in an SQLite database,

In [45]:
from ultralytics import YOLO
import sqlite3
from icecream import ic
import os
import numpy as np
import papermill

In [38]:
MODEL_FILE_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/runs/detect/imgsz9603/weights/best.pt'
IMAGE_DIR_PATH = '/home/aubrey/Desktop/Guam07-training-set/rawdatasubset'
OBJECTS_DB_FILE_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/rawdatasubset.sqlite3'

In [39]:
def detect_objects(model_file_path, image_dir_path):
    ic()
    model = YOLO(MODEL_FILE_PATH)
    results = model(
        source=f'{image_dir_path}/*.jpg',
        imgsz=960,
        agnostic_nms=True,
        iou=0.5,
        stream=True,
        )
    return results

In [40]:
rng = np.random.default_rng(42)

def assign_subset(subset_list=['train', 'val', 'test'], probability_list=[0.8, 0.1, 0.1]):
    ''' 
    Returns member of the subset_list at random using probabilities specified in probability_list.
    This function uses a global random number generator (rng).
    Use "rng = np.random.default_rng(42)" to create rng prior to calling the function.
    '''
    r = rng.multinomial(1, probability_list)
    i = np.where(r == 1)[0][0]
    subset = subset_list[i]
    return subset

# for i in range(20):
#     print(i, assign_subset())

In [41]:
def detected_objects_to_sqlite(detected_objects, objects_db_file_path):
    ic()
    if os.path.exists(objects_db_file_path):
        os.remove(OBJECTS_DB_FILE_PATH)

    conn = sqlite3.connect(objects_db_file_path)
    cur = conn.cursor()

    cur.execute('DROP TABLE IF EXISTS detected_objects')
    table = '''
    create table detected_objects (
        imagepath TEXT not null,
        x REAL not null,
        y REAL not null,
        w REAL not null,
        h REAL not null,
        cls INTEGER not null,
        conf REAL not null,
        subset TEXT NOT NULL,
        xpos INTEGER,
        edited_flag INTEGER DEFAULT 0 NOT NULL
    );
    '''
    cur.execute(table)

    for obj in detected_objects:
        imagepath = obj.path
        boxes = obj.boxes.cpu().numpy()
        class_ids = boxes.cls
        confidences = boxes.conf
        xywhns = boxes.xywhn
        n = len(class_ids)
        subset = assign_subset()
        
        # Calc order of x values for the current image
        # Here, x is the left hand bound of a bounding box
        xlist = []
        for i in range(n):
            x = xywhns[i][0] - xywhns[i][2] / 2
            xlist.append(x)
        sorted_xlist = sorted(xlist)
        index_list = []
        for i in xlist:
            index_list.append(sorted_xlist.index(i))
        
        # Add records for the current image ordered by xpos
        for i in range(n):
            x = xywhns[i][0]    
            y = xywhns[i][1]
            w = xywhns[i][2]
            h = xywhns[i][3]
            cls = int(class_ids[i])
            conf = confidences[i]
            xpos = index_list[i] + 1
            ic(x, xpos)
            cur.execute(f'INSERT INTO detected_objects VALUES ("{imagepath}",{x},{y},{w},{h},{cls},{conf},"{subset}", {xpos}, 0)')
            conn.commit()
            
    conn.close()

In [42]:
# MAIN

detected_objects = detect_objects(MODEL_FILE_PATH, IMAGE_DIR_PATH)

if os.path.exists(OBJECTS_DB_FILE_PATH):
    os.remove(OBJECTS_DB_FILE_PATH)
detected_objects_to_sqlite(detected_objects, OBJECTS_DB_FILE_PATH)

print('FINISHED')

ic| 2396985220.py:2 in detect_objects() at 10:40:11.592
ic| 1429949208.py:2 in detected_objects_to_sqlite() at 10:40:11.683



image 1/63 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111715.jpg: 544x960 1 low, 4.0ms


ic| x: 0.4169082, xpos: 1


image 2/63 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111717.jpg: 544x960 1 low, 3.6ms


ic| x: 0.33176655, xpos: 1


...

image 62/63 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111826.jpg: 544x960 3 mediums, 3.8ms


ic| x: 0.46458974, xpos: 2
ic| x: 0.54915994, xpos: 3
ic| x: 0.42666605, xpos: 1


image 63/63 /home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111827.jpg: 544x960 1 low, 2 mediums, 1 vcut, 3.3ms


ic| x: 0.7846921, xpos: 2
ic| x: 0.90428245, xpos: 4
ic| x: 0.72829354, xpos: 1
ic| x: 0.82961017, xpos: 3


Speed: 1.7ms preprocess, 3.6ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 960)
FINISHED
